# 자연어처리

RNN 을 활용한 텍스트 분류 (Text Classification)

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

-------------------------------
**자연어 처리**<br/><br/>

주어진 `spam.csv` file을 활용하여 스팸 메일 분류기를 만듭니다.

-----------------------------------


## 필요한 모듈 import

In [15]:
import json
import tensorflow as tf
import numpy as np
import urllib
import pandas as pd

pd.set_option('display.max_colwidth', None)

# 필요한 module import

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

## 필요한 데이터 셋 다운로드

In [2]:
# 데이터셋 다운로드
url = 'https://www.dropbox.com/s/iwm9kucrx5i6h2e/spam.csv?dl=1'
urllib.request.urlretrieve(url, 'spam.csv')

('spam.csv', <http.client.HTTPMessage at 0x7f47918bbf50>)

## 데이터 로드

In [3]:
# 데이터셋 로드
df = pd.read_csv('spam.csv', encoding='latin1')
df = df[['v1', 'v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,"This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free


## 전처리: 데이터셋 구성(sentences, labels)

다음과 같이 `v1` 컬럼의 `ham`은 0으로 `spam`은 1로 변환합니다.

In [6]:
# 코드를 입력해 주세요
df['v1'] = df['v1'].apply(lambda x: 1 if x == 'spam' else 0)


In [7]:
df['v1'].value_counts()

0    4825
1     747
Name: v1, dtype: int64

## train / valid 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split

SEED = 123

데이터셋 분할 옵션

- test_size=0.2
- stratify 지정
- random_state=SEED 지정

In [9]:
# train_sentence, valid_sentence, train_label, valid_label = # 코드를 입력해 주세요
# train_test_split (
#     df['v2'],
#     df['v1'],
# )

train_sentence, valid_sentence, train_label, valid_label = train_test_split(df['v2'], df['v1'], stratify=df['v1'], test_size=0.2, random_state=SEED)

In [10]:
train_sentence.shape, valid_sentence.shape

((4457,), (1115,))

In [11]:
train_sentence[:5]

4128                                                                                                                                  Me i'm not workin. Once i get job...
134     Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p
2360                                             Had the money issue weigh me down but thanks to you, I can breathe easier now. I.ll make sure you dont regret it. Thanks.
3545                          SO IS TH GOWER MATE WHICH IS WHERE I AM!?! HOW R U MAN? ALL IS GOOD IN WALES ILL B BACK åÔMORROW. C U THIS WK? WHO WAS THE MSG 4? åÐ RANDOM!
3253                                                                                                                                I can make lasagna for you... vodka...
Name: v2, dtype: object

In [12]:
train_label.shape, valid_label.shape

((4457,), (1115,))

In [13]:
train_label[:5]

4128    0
134     1
2360    0
3545    0
3253    0
Name: v1, dtype: int64

## Tokenizer 정의

단어의 토큰화를 진행합니다.

* `num_words`: 단어 max 사이즈를 지정합니다. 가장 **빈도수가 높은** 단어부터 저장합니다.
* `oov_token`: 단어 토큰에 없는 단어를 어떻게 표기할 것인지 지정해줍니다.

In [16]:
vocab_size = 1000
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

## Tokenizer로 학습시킬 문장에 대한 토큰화 진행

`fit_on_texts`로 학습할 문장에 대하여 **토큰화**를 진행합니다.

In [18]:
tokenizer.fit_on_texts(train_sentence)

In [19]:
for key, value in tokenizer.word_index.items():
    print('{}  \t======>\t {}'.format(key, value))
    if value == 25:
        break

<OOV>  	======>	 1
i  	======>	 2
to  	======>	 3
you  	======>	 4
a  	======>	 5
the  	======>	 6
u  	======>	 7
and  	======>	 8
is  	======>	 9
in  	======>	 10
me  	======>	 11
my  	======>	 12
for  	======>	 13
your  	======>	 14
it  	======>	 15
of  	======>	 16
call  	======>	 17
have  	======>	 18
on  	======>	 19
are  	======>	 20
now  	======>	 21
2  	======>	 22
that  	======>	 23
so  	======>	 24
not  	======>	 25


In [22]:
word_index = tokenizer.word_index
len(tokenizer.word_index), word_index['<OOV>'], word_index['to']

(7907, 1, 3)

## 문장(sentences)을 Sequence로 변환

`texts_to_sequences`: 문장을 숫자로 **치환** 합니다. Train Set, Valid Set 모두 별도로 적용해주어야 합니다.

In [45]:
train_sequences = tokenizer.texts_to_sequences(train_sentence)
validation_sequences = tokenizer.texts_to_sequences(valid_sentence)


In [26]:
train_sequences[0]

[11, 35, 25, 1, 402, 2, 30, 297]

## 시퀀스의 길이를 맞춰주기

3가지 옵션을 입력해 줍니다.

* `maxlen`: 최대 문장 길이를 정의합니다. 최대 문장길이보다 길면, 잘라냅니다.
* `truncating`: 문장의 길이가 `maxlen`보다 길 때 앞을 자를지 뒤를 자를지 정의합니다.
* `padding`: 문장의 길이가 `maxlen`보다 짧을 때 채워줄 값을 앞을 채울지, 뒤를 채울지 정의합니다.

In [29]:
# 한 문장의 최대 단어 숫자
max_length = 120

# 잘라낼 문장의 위치
trunc_type='post'

# 채워줄 문장의 위치
padding_type='post'

train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
train_padded.shape, validation_padded.shape

((4457, 120), (1115, 120))

In [38]:
train_labels = np.array(train_label)
validation_labels = np.array(valid_label)

## 모델 정의 (Sequential)

출력층은 `Dense(1, activation='sigmoid')`로 설정 합니다.

In [32]:
embedding_dim = 16

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

요약 확인

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 120, 128)         41472     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 1

## 컴파일 (compile)

알맞은 `loss`와 `metric`을 설정합니다.

In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

In [35]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

## 학습 (fit)

In [39]:
model.fit(train_padded, train_labels, 
          validation_data=(validation_padded, validation_labels),
          callbacks=[checkpoint],
          epochs=10)

Epoch 1/10
140/140 [==============================] - ETA: 0s - loss: 0.2005 - acc: 0.9316
Epoch 1: val_loss improved from inf to 0.05740, saving model to my_checkpoint.ckpt
140/140 [==============================] - 16s 34ms/step - loss: 0.2005 - acc: 0.9316 - val_loss: 0.0574 - val_acc: 0.9865
Epoch 2/10
139/140 [============================>.] - ETA: 0s - loss: 0.0482 - acc: 0.9870
Epoch 2: val_loss improved from 0.05740 to 0.05575, saving model to my_checkpoint.ckpt
140/140 [==============================] - 3s 23ms/step - loss: 0.0482 - acc: 0.9870 - val_loss: 0.0557 - val_acc: 0.9874
Epoch 3/10
139/140 [============================>.] - ETA: 0s - loss: 0.0309 - acc: 0.9919
Epoch 3: val_loss did not improve from 0.05575
140/140 [==============================] - 3s 23ms/step - loss: 0.0308 - acc: 0.9919 - val_loss: 0.0605 - val_acc: 0.9848
Epoch 4/10
138/140 [============================>.] - ETA: 0s - loss: 0.0619 - acc: 0.9846
Epoch 4: val_loss did not improve from 0.05575
140/1

## 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

In [40]:
model.load_weights(checkpoint_path)

## 모델 평가 (Evaluate)

`model.evaluate()`

In [44]:
model.evaluate(validation_padded,validation_labels)

35/35 [==============================] - 0s 13ms/step - loss: 0.0557 - acc: 0.9874


[0.05574829503893852, 0.9874439239501953]